In [9]:
import pandas as pd

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
pd.options.display.float_format = '{:.2f}'.format

# 사용한 변수 및 가중치 가져오기

In [11]:
df_tmp = pd.read_excel("./변수 및 가중치 입력.xlsx", usecols=['변수', '가중치'])
df_tmp = df_tmp[df_tmp['가중치'] > 0].reset_index(drop=True)   # 가중치가 있는 변수만 추리기
useValsLst = dict(zip(df_tmp['변수'], df_tmp['가중치']))
useValsLst.keys()

dict_keys(['사업체수', '신고사건접수건수', '근로감독건수', '정기감독실시업체', '안전보건진단·개선계획수립명령건수(개선계획)'])

In [12]:
list(useValsLst.keys())

['사업체수', '신고사건접수건수', '근로감독건수', '정기감독실시업체', '안전보건진단·개선계획수립명령건수(개선계획)']

In [13]:
['기관명','과수(전체)','전체정원','감독관'] +  list(useValsLst.keys())

['기관명',
 '과수(전체)',
 '전체정원',
 '감독관',
 '사업체수',
 '신고사건접수건수',
 '근로감독건수',
 '정기감독실시업체',
 '안전보건진단·개선계획수립명령건수(개선계획)']

# 사용자가 입력한 변수만 가져오기

In [15]:
df_org = pd.read_csv("./(250617)근로감독관 충원 RAW DATA(v0.2).csv",  
                     usecols = ['기관명','과수(전체)','전체정원','감독관'] +  list(useValsLst.keys()),
                     encoding = 'cp949')

In [16]:
df_org.head()

,기관명,과수(전체),전체정원,감독관,사업체수,신고사건접수건수,근로감독건수,정기감독실시업체,안전보건진단·개선계획수립명령건수(개선계획)
0,서울청,17,297,146,96342,11789,1357,1136,NaN
1,서울강남지청,8,161,88,60696,12226,865,741,NaN
2,서울동부지청,8,187,82,73619,10836,749,656,NaN
3,서울서부지청,7,176,71,52899,8233,642,582,NaN
4,서울남부지청,8,180,75,60319,9804,745,620,1.00


In [17]:
df_org.dtypes

기관명                         object
과수(전체)                       int64
전체정원                         int64
감독관                          int64
사업체수                         int64
신고사건접수건수                     int64
근로감독건수                       int64
정기감독실시업체                     int64
안전보건진단·개선계획수립명령건수(개선계획)    float64
dtype: object

# 감독관 100명 증원

##### 프로세스 설명 : 1.감독관 1인당 담당량을 계산  2. 1인당 담당량이 가장 만은 지청에 감독관 1명을 증원  3.  1,2 반복


In [18]:
df_증원 = df_org.copy()
df_증원['반복회수'] = 0
df_증원 = df_증원[['반복회수'] + [col for col in df_증원.columns if col != '반복회수']]
plus_arr = []
df_증원log = pd.DataFrame(columns = df_증원.columns)
df_증원기관 = pd.DataFrame(columns = ['반복회차', '기관명', '현감독관', '작업후_감독관'])

In [20]:

scaler = MinMaxScaler()
for val in useValsLst.keys() :
    df_증원['1인당_' + val] = df_증원[val]/df_증원['감독관']
    df_증원['1인당_N_' + val] = scaler.fit_transform(df_증원[['1인당_' + val]])*100 

v = 0
for val in useValsLst.keys() :
    v += df_증원['1인당_N_' + val]
    
df_증원['1인당_업무량'] = v

    
    
    

In [25]:
df_증원.columns


Index(['반복회수', '기관명', '과수(전체)', '전체정원', '감독관', '사업체수', '신고사건접수건수', '근로감독건수',
       '정기감독실시업체', '안전보건진단·개선계획수립명령건수(개선계획)', '1인당_사업체수', '1인당_N_사업체수',
       '1인당_신고사건접수건수', '1인당_N_신고사건접수건수', '1인당_근로감독건수', '1인당_N_근로감독건수',
       '1인당_정기감독실시업체', '1인당_N_정기감독실시업체', '1인당_안전보건진단·개선계획수립명령건수(개선계획)',
       '1인당_N_안전보건진단·개선계획수립명령건수(개선계획)', '1인당_업무량'],
      dtype='object')

In [24]:
for i in range(0,100):

    ## 변수별 감독관 1인당 담당량 계산
    for val in useValsLst.keys() :
    
        df_증원['1인당_인구수'] = df_증원['인구수']/df_증원['감독관']
        df_증원['1인당_사업체수'] = df_증원['사업체수']/df_증원['감독관']
        df_증원['1인당_종사자수'] = df_증원['종사자수']/df_증원['감독관']
        df_증원['1인당_신고사건접수건수'] = df_증원['신고사건접수건수']/df_증원['감독관']

        ### 1인당 총 업무량(Point) 계산
        df_증원['1인당_업무량'] = df_증원['1인당_인구수'] + df_증원['1인당_사업체수'] + df_증원['1인당_종사자수'] + df_증원['1인당_신고사건접수건수']

        ### 1인당 업무량 높은 순 정렬
        df_증원 = df_증원.sort_values(['1인당_업무량'], ascending = False).copy().reset_index(drop=True)
        df_증원['반복회수'] = i
        
        #로그 저장
        df_증원log = pd.concat([df_증원log, df_증원],axis=0)
        tmp = pd.DataFrame({'반복회차' : [i], 
                                '기관명':[df_증원.loc[0, '기관명']], 
                                '현감독관':[df_org[df_org['기관명'] == df_증원.loc[0, '기관명']].reset_index().loc[0, '감독관']],
                                '작업후_감독관': [df_증원.loc[0, '감독관']]})
        df_증원기관 = pd.concat([df_증원기관, tmp], axis=0)


        ### 상위 1번 지청에 감독관 1명 증원
        df_증원.loc[0, '감독관'] += 1
        plus_arr.append(df_증원.loc[0, '기관명'])


KeyError: '인구수'

In [ ]:
df_증원log = df_증원log[df_증원.columns]

In [ ]:
df_증원log.to_excel("반복회차별 데이터.xlsx")
df_증원기관.to_excel("반복회차별 증원된 기관.xlsx")

In [ ]:
#with pd.ExcelWriter("결과물2.xlsx", engine='openpyxl') as writer:
#    df_증원log.to_excel(writer, sheet_name = "반복회차별 데이터", index=False)
#    df_증원기관.to_excel(writer, sheet_name = "반복회차별 증원된 기관", index=False)